# Part III: Technical Analysis – Mastering the Chart

## Chapter 13: Candlestick Patterns

**Chapter Objective:** Candlestick charts are the preferred visual tool of most modern technical analysts because they convey a wealth of information in a single intuitive image. This chapter provides a comprehensive guide to candlestick patterns—from basic single‑session formations to complex multi‑session reversal and continuation signals. You will learn the psychology behind each pattern, how to recognize them reliably, and how to integrate candlestick signals with trend analysis and other technical tools. By mastering candlesticks, you will gain deeper insight into the ongoing battle between buyers and sellers and improve your timing for entries and exits.

---

### 13.1 The Anatomy of a Candlestick

Candlestick charts originated in Japan over 300 years ago, used by rice traders. They were introduced to the Western world by Steve Nison in his book *Japanese Candlestick Charting Techniques*. Each candlestick represents the price action for a specific period (day, week, hour, etc.) and consists of several parts.

**Components of a Candlestick:**

- **Real Body:** The rectangular area between the open and close. It represents the net price movement during the period.
    - If the close is above the open, the body is typically white or green (bullish).
    - If the close is below the open, the body is typically black or red (bearish).
- **Upper Shadow (or Wick):** The thin line extending from the top of the real body to the period's high. It shows the highest price reached.
- **Lower Shadow:** The thin line extending from the bottom of the real body to the period's low. It shows the lowest price reached.

**Interpretation of Body and Shadow Lengths:**

- **Long real body** indicates strong buying (bullish) or selling (bearish) pressure.
- **Short real body** indicates indecision or consolidation (e.g., doji, spinning top).
- **Long upper shadow** suggests that buyers pushed prices up, but sellers forced them back down (rejection of higher prices).
- **Long lower shadow** suggests that sellers pushed prices down, but buyers stepped in to push them back up (rejection of lower prices).

**Python Code Snippet: Plotting Candlesticks with mplfinance**

```python
import yfinance as yf
import mplfinance as mpf

# Download data
ticker = 'AAPL'
data = yf.download(ticker, start='2023-10-01', end='2024-01-01')

# Basic candlestick chart
mpf.plot(data, type='candle', style='yahoo', title=f'{ticker} - Candlestick Chart', volume=True, figsize=(12,8))
```

---

### 13.2 Single‑Session Indicators

Single candlestick patterns are formed by just one period's price action. They provide immediate clues about market sentiment.

#### Doji

A doji occurs when the open and close are virtually equal, resulting in a very small real body. It signals indecision between buyers and sellers.

**Types of Doji:**

- **Standard Doji:** Open and close equal, with shadows of any length.
- **Long‑Legged Doji:** Long upper and lower shadows, indicating that price moved significantly in both directions but closed near the open—extreme indecision.
- **Dragonfly Doji:** Long lower shadow, little or no upper shadow. Forms at the bottom of a downtrend, suggesting a potential reversal up (buyers stepped in after a sell‑off).
- **Gravestone Doji:** Long upper shadow, little or no lower shadow. Forms at the top of an uptrend, suggesting a potential reversal down (sellers stepped in after a rally).

**Interpretation:**
- A doji after a strong uptrend warns of a possible top.
- A doji after a strong downtrend warns of a possible bottom.
- Doji are more significant when they appear after a prolonged trend and when accompanied by high volume.

#### Marubozu

A marubozu (Japanese for "bald head") has no shadows (or very small ones), meaning the open was the low (for a bullish marubozu) or the high (for a bearish marubozu).

- **White Marubozu:** Long white body with no lower shadow (open = low) and no upper shadow (close = high). Indicates strong buying pressure from open to close.
- **Black Marubozu:** Long black body with no upper shadow (open = high) and no lower shadow (close = low). Indicates strong selling pressure.

#### Hammer and Hanging Man

These patterns have the same shape: a small real body at the upper end of the trading range, with a long lower shadow (at least twice the length of the body) and little or no upper shadow. They are distinguished by their location in the trend.

- **Hammer:** Appears after a downtrend. The long lower shadow suggests that sellers pushed prices down, but buyers overwhelmed them and pushed prices back up, closing near the high. Bullish reversal signal.
- **Hanging Man:** Appears after an uptrend. The same shape, but after a rally, it warns that selling pressure is emerging. Bearish reversal signal.

#### Inverted Hammer and Shooting Star

These patterns have a small real body at the lower end of the trading range, with a long upper shadow and little or no lower shadow.

- **Inverted Hammer:** Appears after a downtrend. The long upper shadow indicates that buyers tried to push prices up, but sellers still have some control. However, it shows that buying interest is emerging. Needs confirmation on the next bar.
- **Shooting Star:** Appears after an uptrend. The long upper shadow shows that buyers pushed prices higher, but sellers drove them back down, closing near the low. Bearish reversal signal.

**Python Code Snippet: Detecting Single Candlestick Patterns**

```python
def detect_single_candle_patterns(df):
    """
    Detects common single candlestick patterns.
    Returns a DataFrame with pattern names.
    """
    patterns = pd.DataFrame(index=df.index)
    patterns['Doji'] = abs(df['Close'] - df['Open']) <= (df['High'] - df['Low']) * 0.1  # small body relative to range

    # Hammer: after downtrend, small body at top, long lower shadow
    # We'll need trend context; here just shape detection
    body = abs(df['Close'] - df['Open'])
    lower_shadow = df['Open'].where(df['Close'] > df['Open'], df['Close']) - df['Low']  # low to min(open,close)
    upper_shadow = df['High'] - df['Open'].where(df['Close'] > df['Open'], df['Close'])  # max(open,close) to high

    patterns['Hammer_Shape'] = (lower_shadow >= 2 * body) & (upper_shadow <= body * 0.3)
    patterns['ShootingStar_Shape'] = (upper_shadow >= 2 * body) & (lower_shadow <= body * 0.3)

    # Marubozu: very small shadows
    patterns['White_Marubozu'] = (body > (df['High'] - df['Low']) * 0.7) & (df['Close'] > df['Open']) & (lower_shadow <= body * 0.1) & (upper_shadow <= body * 0.1)
    patterns['Black_Marubozu'] = (body > (df['High'] - df['Low']) * 0.7) & (df['Close'] < df['Open']) & (lower_shadow <= body * 0.1) & (upper_shadow <= body * 0.1)

    return patterns

patterns_single = detect_single_candle_patterns(data)
print("Single candlestick patterns detected (last 5 days):")
print(patterns_single.tail())
```

---

### 13.3 Two‑Session Indicators

Two‑session patterns involve two consecutive candlesticks. They are more reliable than single‑session patterns because they show a shift in sentiment over two periods.

#### Bullish Engulfing

A two‑candle reversal pattern that occurs in a downtrend.

- **Day 1:** A bearish (black/red) candle, continuing the downtrend.
- **Day 2:** A bullish (white/green) candle that opens lower than the previous close but closes above the previous open, i.e., the real body completely "engulfs" the previous day's real body.
- **Interpretation:** The selling pressure is overcome by stronger buying pressure. A strong bullish reversal signal, especially if followed by higher prices on day 3.

#### Bearish Engulfing

The opposite, occurring in an uptrend.

- **Day 1:** A bullish candle.
- **Day 2:** A bearish candle that opens higher than the previous close and closes below the previous open, engulfing the prior body.
- **Interpretation:** Strong bearish reversal signal.

#### Harami (and Harami Cross)

Harami means "pregnant" in Japanese. It is the opposite of engulfing.

- **Bullish Harami:** Occurs in a downtrend. Day 1 is a long bearish candle; Day 2 is a small bullish candle (or doji) that trades entirely within the previous day's real body. It suggests that selling pressure is abating.
- **Bearish Harami:** Occurs in an uptrend. Day 1 is a long bullish candle; Day 2 is a small bearish candle within the prior body. Suggests weakening of buying pressure.
- **Harami Cross:** When the second day is a doji. More significant than a regular harami.

#### Piercing Line and Dark Cloud Cover

- **Piercing Line (Bullish):** In a downtrend. Day 1: bearish candle. Day 2: bullish candle that opens lower but closes above the midpoint of Day 1's real body. Shows strong buying pressure overcoming selling.
- **Dark Cloud Cover (Bearish):** In an uptrend. Day 1: bullish candle. Day 2: bearish candle that opens higher but closes below the midpoint of Day 1's real body. Shows strong selling pressure.

#### Tweezer Tops and Bottoms

- **Tweezer Top:** Two consecutive candles with the same high. In an uptrend, it signals resistance and potential reversal.
- **Tweezer Bottom:** Two consecutive candles with the same low. In a downtrend, it signals support and potential reversal.

**Python Code Snippet: Detecting Two‑Candle Patterns**

```python
def detect_two_candle_patterns(df):
    patterns = pd.DataFrame(index=df.index[1:])  # start from second day

    # Bullish Engulfing
    patterns['Bullish_Engulfing'] = (
        (df['Close'].shift(1) < df['Open'].shift(1)) &  # Day1 bearish
        (df['Close'] > df['Open']) &                     # Day2 bullish
        (df['Open'] < df['Close'].shift(1)) &            # Day2 open below Day1 close
        (df['Close'] > df['Open'].shift(1)) &            # Day2 close above Day1 open
        (abs(df['Close'] - df['Open']) > abs(df['Close'].shift(1) - df['Open'].shift(1)))  # Day2 body larger
    )

    # Bearish Engulfing
    patterns['Bearish_Engulfing'] = (
        (df['Close'].shift(1) > df['Open'].shift(1)) &  # Day1 bullish
        (df['Close'] < df['Open']) &                     # Day2 bearish
        (df['Open'] > df['Close'].shift(1)) &            # Day2 open above Day1 close
        (df['Close'] < df['Open'].shift(1)) &            # Day2 close below Day1 open
        (abs(df['Close'] - df['Open']) > abs(df['Close'].shift(1) - df['Open'].shift(1)))
    )

    # Bullish Harami
    patterns['Bullish_Harami'] = (
        (df['Close'].shift(1) < df['Open'].shift(1)) &  # Day1 bearish
        (df['Close'] > df['Open']) &                     # Day2 bullish
        (df['Open'] > df['Close'].shift(1)) &            # Day2 open above Day1 close
        (df['Close'] < df['Open'].shift(1)) &            # Day2 close below Day1 open
        (abs(df['Close'] - df['Open']) < abs(df['Close'].shift(1) - df['Open'].shift(1)))  # Day2 body smaller
    )

    # Bearish Harami
    patterns['Bearish_Harami'] = (
        (df['Close'].shift(1) > df['Open'].shift(1)) &  # Day1 bullish
        (df['Close'] < df['Open']) &                     # Day2 bearish
        (df['Open'] < df['Close'].shift(1)) &            # Day2 open below Day1 close
        (df['Close'] > df['Open'].shift(1)) &            # Day2 close above Day1 open
        (abs(df['Close'] - df['Open']) < abs(df['Close'].shift(1) - df['Open'].shift(1)))
    )

    # Piercing Line
    patterns['Piercing_Line'] = (
        (df['Close'].shift(1) < df['Open'].shift(1)) &  # Day1 bearish
        (df['Close'] > df['Open']) &                     # Day2 bullish
        (df['Open'] < df['Low'].shift(1)) &              # Day2 open below Day1 low
        (df['Close'] > (df['Open'].shift(1) + df['Close'].shift(1)) / 2) &  # Day2 close above midpoint of Day1 body
        (df['Close'] < df['Open'].shift(1))              # Day2 close below Day1 open
    )

    # Dark Cloud Cover
    patterns['Dark_Cloud'] = (
        (df['Close'].shift(1) > df['Open'].shift(1)) &  # Day1 bullish
        (df['Close'] < df['Open']) &                     # Day2 bearish
        (df['Open'] > df['High'].shift(1)) &             # Day2 open above Day1 high
        (df['Close'] < (df['Open'].shift(1) + df['Close'].shift(1)) / 2) &  # Day2 close below midpoint
        (df['Close'] > df['Open'].shift(1))              # Day2 close above Day1 open (still above)
    )

    return patterns

patterns_two = detect_two_candle_patterns(data)
print("\nTwo‑candle patterns detected (last 5 days):")
print(patterns_two.tail())
```

---

### 13.4 Three‑Session Indicators

Three‑candle patterns provide even stronger signals because they show a clear shift in sentiment over several periods.

#### Morning Star

A three‑candle bullish reversal pattern that appears after a downtrend.

- **Day 1:** A long bearish candle (continuation of downtrend).
- **Day 2:** A small‑bodied candle (doji or spinning top) that gaps down from Day 1. It shows indecision.
- **Day 3:** A long bullish candle that closes well into the body of Day 1 (ideally above the midpoint). Confirms reversal.
- **Interpretation:** Strong bullish signal, especially if Day 2 is a doji (morning doji star).

#### Evening Star

The bearish equivalent, appearing after an uptrend.

- **Day 1:** A long bullish candle.
- **Day 2:** A small‑bodied candle that gaps up from Day 1 (indecision).
- **Day 3:** A long bearish candle that closes well into the body of Day 1.
- **Interpretation:** Strong bearish reversal signal.

#### Three White Soldiers

Three consecutive long bullish candles, each closing near its high and opening within the previous candle's body. Indicates strong sustained buying pressure. Usually occurs after a downtrend or consolidation and signals a strong uptrend.

#### Three Black Crows

Three consecutive long bearish candles, each closing near its low and opening within the previous candle's body. Signals strong selling pressure and a bearish reversal.

#### Abandoned Baby

A rare and powerful reversal pattern.

- **Bullish Abandoned Baby:** In a downtrend, Day 1 is a bearish candle, Day 2 gaps down and forms a doji, Day 3 gaps up and forms a bullish candle. The gaps on both sides of the doji create an "island."
- **Bearish Abandoned Baby:** In an uptrend, Day 1 bullish, Day 2 gaps up and forms a doji, Day 3 gaps down and forms a bearish candle.

**Python Code Snippet: Detecting Three‑Candle Patterns**

```python
def detect_three_candle_patterns(df):
    patterns = pd.DataFrame(index=df.index[2:])  # start from third day

    # Morning Star
    patterns['Morning_Star'] = (
        (df['Close'].shift(2) < df['Open'].shift(2)) &  # Day1 bearish
        (abs(df['Close'].shift(1) - df['Open'].shift(1)) <= (df['High'].shift(1) - df['Low'].shift(1)) * 0.3) &  # Day2 small body
        (df['Close'].shift(1) < df['Close'].shift(2)) &  # Day2 gaps down (simplified)
        (df['Close'] > df['Open']) &                     # Day3 bullish
        (df['Close'] > (df['Open'].shift(2) + df['Close'].shift(2)) / 2)  # Day3 closes above midpoint of Day1 body
    )

    # Evening Star
    patterns['Evening_Star'] = (
        (df['Close'].shift(2) > df['Open'].shift(2)) &  # Day1 bullish
        (abs(df['Close'].shift(1) - df['Open'].shift(1)) <= (df['High'].shift(1) - df['Low'].shift(1)) * 0.3) &  # Day2 small body
        (df['Close'].shift(1) > df['Close'].shift(2)) &  # Day2 gaps up (simplified)
        (df['Close'] < df['Open']) &                     # Day3 bearish
        (df['Close'] < (df['Open'].shift(2) + df['Close'].shift(2)) / 2)  # Day3 closes below midpoint
    )

    # Three White Soldiers
    patterns['Three_White_Soldiers'] = (
        (df['Close'].shift(2) > df['Open'].shift(2)) &  # Day1 bullish
        (df['Close'].shift(1) > df['Open'].shift(1)) &  # Day2 bullish
        (df['Close'] > df['Open']) &                     # Day3 bullish
        (df['Close'].shift(2) > df['Open'].shift(2) * 1.01) &  # bodies are substantial
        (df['Close'].shift(1) > df['Open'].shift(1) * 1.01) &
        (df['Close'] > df['Open'] * 1.01) &
        (df['Open'].shift(1) > df['Open'].shift(2)) &   # opens progressively higher
        (df['Open'] > df['Open'].shift(1))
    )

    # Three Black Crows
    patterns['Three_Black_Crows'] = (
        (df['Close'].shift(2) < df['Open'].shift(2)) &  # Day1 bearish
        (df['Close'].shift(1) < df['Open'].shift(1)) &  # Day2 bearish
        (df['Close'] < df['Open']) &                     # Day3 bearish
        (df['Close'].shift(2) < df['Open'].shift(2) * 0.99) &
        (df['Close'].shift(1) < df['Open'].shift(1) * 0.99) &
        (df['Close'] < df['Open'] * 0.99) &
        (df['Open'].shift(1) < df['Open'].shift(2)) &   # opens progressively lower
        (df['Open'] < df['Open'].shift(1))
    )

    return patterns

patterns_three = detect_three_candle_patterns(data)
print("\nThree‑candle patterns detected (last 5 days):")
print(patterns_three.tail())
```

---

### 13.5 Integrating Candlesticks with Trend Analysis

Candlestick patterns are most powerful when combined with the broader trend context. A hammer at the end of a downtrend is a much stronger signal than a hammer in the middle of a trading range. Similarly, an engulfing pattern in an overextended market (based on RSI or Bollinger Bands) carries more weight.

**Guidelines for Integration:**

- **Always consider the trend.** Use moving averages, trendlines, or higher time frame analysis to determine the dominant trend. Trade reversal patterns only when they align with a potential trend change.
- **Look for confirmation.** A single candlestick pattern is not a signal to act; wait for the next candle to confirm (e.g., after a hammer, wait for a bullish close or a higher open).
- **Combine with support/resistance.** A bullish reversal pattern at a major support level is more reliable.
- **Use volume.** Patterns accompanied by high volume are more significant. For example, a bullish engulfing on heavy volume is a stronger signal.
- **Check momentum indicators.** If RSI shows bullish divergence and a morning star appears, the signal is reinforced.

**Python Code Snippet: Combining Candlesticks with Trend and Support**

```python
def enhanced_signal(df):
    # Identify recent support/resistance (simplified: recent low)
    recent_low = df['Low'].rolling(20).min()
    recent_high = df['High'].rolling(20).max()

    # Trend: use 50-day SMA
    df['SMA50'] = df['Close'].rolling(50).mean()
    df['Trend'] = np.where(df['Close'] > df['SMA50'], 'Uptrend', 'Downtrend')

    # Detect patterns (we'll use a simplified function)
    patterns = detect_three_candle_patterns(df)

    # Combine: morning star in downtrend near recent low
    signals = pd.DataFrame(index=patterns.index)
    signals['Morning_Star_Signal'] = (
        patterns['Morning_Star'] &
        (df.loc[patterns.index, 'Trend'] == 'Downtrend') &
        (df.loc[patterns.index, 'Low'] <= recent_low.loc[patterns.index] * 1.02)  # near recent low
    )

    # Evening star in uptrend near recent high
    signals['Evening_Star_Signal'] = (
        patterns['Evening_Star'] &
        (df.loc[patterns.index, 'Trend'] == 'Uptrend') &
        (df.loc[patterns.index, 'High'] >= recent_high.loc[patterns.index] * 0.98)
    )

    return signals

signals = enhanced_signal(data)
print("\nEnhanced signals (last 5 days):")
print(signals.tail())
```

---

### 13.6 Practical Application: A Step‑by‑Step Candlestick Analysis

Let's walk through a real example using Apple daily data.

**Step 1: Identify the Overall Trend**

Check the weekly chart or use a 200‑day moving average. Assume the primary trend is up.

**Step 2: Look for Reversal Patterns at Key Levels**

Scan for bearish reversal patterns (shooting star, bearish engulfing, evening star) near resistance or after a prolonged rally.

**Step 3: Look for Continuation Patterns**

In a strong uptrend, patterns like the three white soldiers confirm the trend.

**Step 4: Check for Confirmation**

After a hammer, wait for the next day to see if price moves higher. After a bearish engulfing, wait for a lower close.

**Step 5: Combine with Other Tools**

If a bullish engulfing appears at support with RSI oversold, the signal is strong.

**Step 6: Make a Trading Decision**

- **Long Entry:** Bullish reversal pattern confirmed, with volume, at support, and in the context of a primary uptrend.
- **Short Entry:** Bearish reversal pattern confirmed, at resistance, in a primary downtrend (or for a counter‑trend trade with tight stops).

**Python Code Snippet: Visualizing Candlestick Patterns on a Chart**

```python
# Use mplfinance to add pattern markers
# We'll create a custom plot with markers for detected patterns

import mplfinance as mpf

# Example: mark morning stars
patterns = detect_three_candle_patterns(data)
morning_stars = patterns[patterns['Morning_Star']].index

# Create a list of tuples (date, note) for markers
apds = []
for ms in morning_stars:
    apds.append(mpf.make_addplot(data.loc[ms:ms, 'Low']*0.99, type='scatter', markersize=200, marker='^', color='g'))

mpf.plot(data, type='candle', style='yahoo', addplot=apds, title=f'{ticker} with Morning Stars', figsize=(12,8))
```

---

### Chapter Summary

- **Candlesticks provide a visual representation of the battle between buyers and sellers.** Each candle shows open, high, low, and close.
- **Single‑session patterns** (doji, hammer, shooting star, marubozu) give immediate clues about sentiment and potential reversals.
- **Two‑session patterns** (engulfing, harami, piercing, dark cloud) show a shift in sentiment over two periods and are more reliable.
- **Three‑session patterns** (morning/evening star, three soldiers/crows, abandoned baby) provide strong reversal signals.
- **Patterns must be interpreted in the context of the prevailing trend, support/resistance, and volume.** They are not standalone signals.
- **Confirmation is key.** Wait for the next candle to confirm the pattern's implication.
- **Combine candlestick patterns with other technical tools** (moving averages, RSI, Bollinger Bands) for higher probability setups.

**Exercises:**

1.  **Conceptual:** Draw a hammer and a shooting star. Explain the difference in their location within a trend and what each implies about market psychology.
2.  **Practical:** Download daily data for a stock of your choice. Using the pattern detection code, identify all bullish engulfing patterns that occurred in the last year. For each, check whether the stock was higher 5 days later. What is your success rate?
3.  **Research:** Find a real chart where a morning star pattern formed. Measure the subsequent price move. Was the pattern followed by a sustained reversal or just a short‑term bounce? What other factors (volume, trend, support) were present?
4.  **Coding:** Enhance the pattern detection functions to incorporate a trend filter. For example, only flag a bullish engulfing if the stock is in a downtrend (price below 50‑day SMA). Then backtest this filtered strategy against the unfiltered version. Does filtering improve performance?

---

**Looking Ahead to Chapter 14: Advanced Technical Concepts**

Candlestick patterns, combined with the indicators from Chapter 12, provide a robust technical toolkit. However, there are even more advanced theories that some traders use to gain an edge. In Chapter 14, we will explore Elliott Wave Theory, Fibonacci retracements and extensions, Point and Figure charting, and the critical concept of confirmation. These advanced tools can help you anticipate price targets and understand the underlying structure of market movements.